In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum, round

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-14381331-9189-4975-aefb-0268923d916b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 594ms :: artifacts dl 16ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [28]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/speed_distance/"
#to_path_silver = "/projeto/fisico/gold/speed_distance/"

client.delete(to_path,True)
#client.delete(to_path_silver,True)

True

In [2]:
# read data from the silver tables
df = spark.table("silver.speed_distance")

In [12]:
#df.show(truncate=False)
df.printSchema()
df.toPandas()

root
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- AVG_MIN_PER_GAME: double (nullable = true)
 |-- DIST_FEET: double (nullable = true)
 |-- DIST_MILES: double (nullable = true)
 |-- DIST_MILES_OFF: double (nullable = true)
 |-- DIST_MILES_DEF: double (nullable = true)
 |-- DIST_KM: double (nullable = true)
 |-- DIST_KM_OFF: double (nullable = true)
 |-- DIST_KM_DEF: double (nullable = true)
 |-- DIST_KM_PER_MIN: double (nullable = true)
 |-- DIST_KM_OFF_PER_MIN: double (nullable = true)
 |-- DIST_KM_DEF_PER_MIN: double (nullable = true)
 |-- AVG_SPEED_MPH: double (nullable = true)
 |-- AVG_SPEED_MPH_OFF: double (nullable = true)
 |-- AVG_SPEED_MPH_DEF: double (nullable = true)
 |-- AVG_SPEED_KMH: double (nullable = true)
 |-- AVG_SPEED_KMH_OFF: double (nullable = true)
 |-- AVG_SPEED_KMH_DEF: double (nullable = true)

,PLAYER,TEAM,GP,W,L,MIN,AVG_MIN_PER_GAME,DIST_FEET,DIST_MILES,DIST_MILES_OFF,...,AVG_SPEED_MPH,AVG_SPEED_MPH_OFF,AVG_SPEED_MPH_DEF,AVG_SPEED_KMH,AVG_SPEED_KMH_OFF,AVG_SPEED_KMH_DEF,AVG_SPEED_KMH_PER_MIN,AVG_SPEED_KMH_OFF_PER_MIN,AVG_SPEED_KMH_DEF_PER_MIN,SEASON
0,Aaron Brooks,MIN,32,19,13,189,5.906250,71737.0,13.0,7.3,...,4.32,4.46,4.17,6.952366,7.177674,6.710964,0.036785,0.037977,0.035508,2017/2018
1,Aaron Gordon,ORL,58,19,39,1909,32.913793,741938.0,140.0,78.5,...,4.46,5.05,3.88,7.177674,8.127187,6.244255,0.003760,0.004257,0.003271,2017/2018
2,Aaron Harrison,DAL,9,2,7,233,25.888889,87649.0,16.0,9.0,...,4.29,4.52,4.04,6.904086,7.274235,6.501750,0.029631,0.031220,0.027905,2017/2018
3,Aaron Jackson,HOU,1,0,1,35,35.000000,12530.0,2.0,1.2,...,3.94,4.14,3.77,6.340815,6.662684,6.067227,0.181166,0.190362,0.173349,2017/2018
4,Abdel Nader,BOS,47,32,15,520,11.063830,205950.0,39.0,20.9,...,4.50,4.61,4.39,7.242048,7.419076,7.065020,0.013927,0.014267,0.013587,2017/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,Willie Reed,BKN,39,8,31,426,10.923077,158201.0,30.0,16.0,...,4.24,4.45,4.02,6.823619,7.161581,6.469563,0.016018,0.016811,0.015187,2015/2016
4027,Xavier Munford,MEM,14,3,11,244,17.428571,98364.0,18.0,10.1,...,4.57,4.93,4.21,7.354702,7.934066,6.775338,0.030142,0.032517,0.027768,2015/2016
4028,Zach LaVine,MIN,82,29,53,2293,27.963415,880772.0,166.0,87.3,...,4.37,4.62,4.13,7.032833,7.435169,6.646591,0.003067,0.003243,0.002899,2015/2016
4029,Zach Randolph,MEM,68,37,31,2016,29.647059,684888.0,129.0,68.4,...,3.87,3.93,3.80,6.228161,6.324722,6.115507,0.003089,0.003137,0.003033,2015/2016


In [26]:
import pyspark.sql.functions as func

gd_df = df \
    .withColumn("AVG_MIN_PER_GAME", func.round(df["AVG_MIN_PER_GAME"], 2)) \
    .withColumn("AVG_SPEED_KMH", func.round(df["AVG_SPEED_KMH"], 2)) \
    .withColumn("AVG_SPEED_KMH_OFF", func.round(df["AVG_SPEED_KMH_OFF"], 2)) \
    .withColumn("AVG_SPEED_KMH_DEF", func.round(df["AVG_SPEED_KMH_DEF"], 2)) \
    .withColumn("AVG_SPEED_KMH_PER_MIN", func.round(df["AVG_SPEED_KMH_PER_MIN"], 2)) \
    .withColumn("AVG_SPEED_KMH_OFF_PER_MIN", func.round(df["AVG_SPEED_KMH_OFF_PER_MIN"], 2)) \
    .withColumn("AVG_SPEED_KMH_DEF_PER_MIN", func.round(df["AVG_SPEED_KMH_DEF_PER_MIN"], 2)) \
    .withColumn("DIST_KM", func.round(df["DIST_KM"], 2)) \
    .withColumn("DIST_KM_OFF", func.round(df["DIST_KM_OFF"], 2)) \
    .withColumn("DIST_KM_DEF", func.round(df["DIST_KM_DEF"], 2)) \
    .withColumn("DIST_KM_PER_MIN", func.round(df["DIST_KM_PER_MIN"], 2)) \
    .withColumn("DIST_KM_OFF_PER_MIN", func.round(df["DIST_KM_OFF_PER_MIN"], 2)) \
    .withColumn("DIST_KM_DEF_PER_MIN", func.round(df["DIST_KM_DEF_PER_MIN"], 2))
gd_df.show()

+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+-----------------+-------------+-----------------+-----------------+---------------------+-------------------------+-------------------------+---------+
|          PLAYER|TEAM| GP|  W|  L| MIN|AVG_MIN_PER_GAME|DIST_FEET|DIST_MILES|DIST_MILES_OFF|DIST_MILES_DEF|DIST_KM|DIST_KM_OFF|DIST_KM_DEF|DIST_KM_PER_MIN|DIST_KM_OFF_PER_MIN|DIST_KM_DEF_PER_MIN|AVG_SPEED_MPH|AVG_SPEED_MPH_OFF|AVG_SPEED_MPH_DEF|AVG_SPEED_KMH|AVG_SPEED_KMH_OFF|AVG_SPEED_KMH_DEF|AVG_SPEED_KMH_PER_MIN|AVG_SPEED_KMH_OFF_PER_MIN|AVG_SPEED_KMH_DEF_PER_MIN|   SEASON|
+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+----------

In [25]:
#gd_df.show()
gd_df.toPandas()
gd_df.printSchema()

root
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- W: integer (nullable = true)
 |-- L: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- AVG_MIN_PER_GAME: double (nullable = true)
 |-- DIST_FEET: double (nullable = true)
 |-- DIST_MILES: double (nullable = true)
 |-- DIST_MILES_OFF: double (nullable = true)
 |-- DIST_MILES_DEF: double (nullable = true)
 |-- DIST_KM: double (nullable = true)
 |-- DIST_KM_OFF: double (nullable = true)
 |-- DIST_KM_DEF: double (nullable = true)
 |-- DIST_KM_PER_MIN: double (nullable = true)
 |-- DIST_KM_OFF_PER_MIN: double (nullable = true)
 |-- DIST_KM_DEF_PER_MIN: double (nullable = true)
 |-- AVG_SPEED_MPH: double (nullable = true)
 |-- AVG_SPEED_MPH_OFF: double (nullable = true)
 |-- AVG_SPEED_MPH_DEF: double (nullable = true)
 |-- AVG_SPEED_KMH: double (nullable = true)
 |-- AVG_SPEED_KMH_OFF: double (nullable = true)
 |-- AVG_SPEED_KMH_DEF: double (nullable = true)

In [29]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/speed_distance/")

22/01/13 00:14:32 ERROR DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=hdfs://hdfs-nn:9000/projeto/fisico/gold/speed_distance/_delta_log, version=0, metadata=Metadata(2642d87b-57d1-44db-b300-0696674c7ade,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"PLAYER","type":"string","nullable":true,"metadata":{"__CHAR_VARCHAR_TYPE_STRING":"varchar(50)"}},{"name":"TEAM","type":"string","nullable":true,"metadata":{"__CHAR_VARCHAR_TYPE_STRING":"varchar(50)"}},{"name":"GP","type":"integer","nullable":true,"metadata":{}},{"name":"W","type":"integer","nullable":true,"metadata":{}},{"name":"L","type":"integer","nullable":true,"metadata":{}},{"name":"MIN","type":"integer","nullable":true,"metadata":{}},{"name":"AVG_MIN_PER_GAME","type":"double","nullable":true,"metadata":{}},{"name":"DIST_FEET","type":"double","nullable":true,"metadata":{}},{"name":"DIST_MILES","type":"double","nullable":true,"metadata":{}},{"name":"DIST_MILES_

In [22]:
# check the results in the table
spark.table("gold.speed_distance").show()
spark.table("gold.speed_distance").count()

+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+-----------------+-------------+-----------------+-----------------+---------------------+-------------------------+-------------------------+---------+
|          PLAYER|TEAM| GP|  W|  L| MIN|AVG_MIN_PER_GAME|DIST_FEET|DIST_MILES|DIST_MILES_OFF|DIST_MILES_DEF|DIST_KM|DIST_KM_OFF|DIST_KM_DEF|DIST_KM_PER_MIN|DIST_KM_OFF_PER_MIN|DIST_KM_DEF_PER_MIN|AVG_SPEED_MPH|AVG_SPEED_MPH_OFF|AVG_SPEED_MPH_DEF|AVG_SPEED_KMH|AVG_SPEED_KMH_OFF|AVG_SPEED_KMH_DEF|AVG_SPEED_KMH_PER_MIN|AVG_SPEED_KMH_OFF_PER_MIN|AVG_SPEED_KMH_DEF_PER_MIN|   SEASON|
+----------------+----+---+---+---+----+----------------+---------+----------+--------------+--------------+-------+-----------+-----------+---------------+-------------------+-------------------+-------------+-----------------+----------

4031